In [1]:
import datamart_profiler
import io
import pandas as pd

In [2]:
metadata = datamart_profiler.process_dataset("datasets/Boat_Launch_Sites_by_State_Parks_or_Marine_Facility.csv")

In [3]:
metadata.keys()

dict_keys(['size', 'nb_rows', 'average_row_size', 'nb_profiled_rows', 'nb_columns', 'columns', 'nb_spatial_columns', 'nb_categorical_columns', 'nb_numerical_columns', 'types', 'spatial_coverage', 'attribute_keywords'])

In [17]:
ranges = metadata['spatial_coverage'][0]['ranges']

In [16]:
metadata['spatial_coverage'][1]['ranges']

[{'range': {'type': 'envelope',
   'coordinates': [[-79.134885, 43.36913472], [-76.60976708, 42.05885118]]}},
 {'range': {'type': 'envelope',
   'coordinates': [[-76.25514321, 44.93159446], [-73.38722476, 42.81858875]]}},
 {'range': {'type': 'envelope',
   'coordinates': [[-74.4611211, 43.05354647], [-73.2316962, 40.682314]]}}]

In [37]:
from math import radians, sin, cos, sqrt,asin

def haversine_distance(lat1, lon1, lat2, lon2):
    R = 3959.87433 # this is in miles.  For Earth radius in kilometers use 6372.8 km
    dLat = radians(lat2 - lat1)
    dLon = radians(lon2 - lon1)
    lat1 = radians(lat1)
    lat2 = radians(lat2)

    a = sin(dLat/2)**2 + cos(lat1)*cos(lat2)*sin(dLon/2)**2
    c = 2*asin(sqrt(a))

    return R * c

In [38]:
from geopy.geocoders import Nominatim

def calculate_encapsulating_area(coordinates_list):
    min_latitude = float('inf')
    max_latitude = float('-inf')
    min_longitude = float('inf')
    max_longitude = float('-inf')
    geolocator = Nominatim(user_agent="my-app")  # Specify your user agent


    for item in coordinates_list:
        coordinates = item['range']['coordinates']
        min_lon,min_lat = coordinates[0]
        max_lon,max_lat = coordinates[1]

        min_latitude = min(min_latitude, min_lat)
        max_latitude = max(max_latitude, max_lat)
        min_longitude = min(min_longitude, min_lon)
        max_longitude = max(max_longitude, max_lon)

    location = geolocator.reverse((min_latitude, min_longitude), exactly_one=True)
    min_address = location.raw['address']
    min_city = min_address.get('city', '') or min_address.get('town', '') or min_address.get('village', '') or min_address.get('state', '')

    # Reverse geocode the maximum latitude and longitude
    location = geolocator.reverse((max_latitude, max_longitude), exactly_one=True)
    max_address = location.raw['address']
    max_city = max_address.get('city', '') or max_address.get('town', '') or max_address.get('village', '') or max_address.get('state', '')

    area = (max_latitude - min_latitude) * (max_longitude - min_longitude)

    return area,min_city,max_city

In [44]:
def unstack_ranges(ranges):
    coordinates_list = []
    for item in ranges:
        coordinates_list.append(item['range']['coordinates'][0])
        coordinates_list.append(item['range']['coordinates'][1])
    return coordinates_list

def find_furthest_pairs(coords_list):
    max_distance = 0
    furthest_pairs = None
    geolocator = Nominatim(user_agent="my-app")  # Specify your user agent

    coords_list= unstack_ranges(coords_list)
    furthest_pairs = [coords_list[0],coords_list[1]]
    furthestDist = haversine_distance(coords_list[0][1], coords_list[0][0], coords_list[1][1], coords_list[1][0])
    
    distFlag=0
    for item in coords_list[2:]:
        dist1 = haversine_distance(furthest_pairs[0][1], furthest_pairs[0][0], item[1], item[0])
        dist2 = haversine_distance(furthest_pairs[1][1], furthest_pairs[1][0], item[1], item[0])
        if dist1<dist2:
            distFlag=1

        if distFlag==0 and dist1>furthestDist:
            furthestDist = dist1
            furthest_pairs[1] = item
        elif distFlag==1 and dist2>furthestDist:
            furthestDist = dist2
            furthest_pairs[0] = item
        distFlag=0
    
    location = geolocator.reverse((furthest_pairs[0][1],furthest_pairs[0][0]), exactly_one=True)
    min_address = location.raw['address']
    min_city = min_address.get('city', '') or min_address.get('town', '') or min_address.get('village', '') or min_address.get('state', '')

    # Reverse geocode the maximum latitude and longitude
    location = geolocator.reverse((furthest_pairs[1][1],furthest_pairs[1][0]), exactly_one=True)
    max_address = location.raw['address']
    max_city = max_address.get('city', '') or max_address.get('town', '') or max_address.get('village', '') or max_address.get('state', '')


    return min_city,max_city

In [46]:
find_furthest_pairs(ranges)

('Drummond-North Elmsley', 'New York')

In [68]:
metadata = datamart_profiler.process_dataset("datasets/yellow.csv")

In [69]:
metadata.keys()

dict_keys(['size', 'nb_rows', 'average_row_size', 'nb_profiled_rows', 'nb_columns', 'columns', 'nb_temporal_columns', 'nb_numerical_columns', 'types', 'temporal_coverage', 'attribute_keywords'])

In [70]:
metadata['temporal_coverage']

[{'type': 'datetime',
  'column_names': ['tpep_pickup_datetime'],
  'column_indexes': [0],
  'column_types': ['http://schema.org/DateTime'],
  'ranges': [{'range': {'gte': 1499119232.0, 'lte': 1503824384.0}},
   {'range': {'gte': 1504346368.0, 'lte': 1509134336.0}},
   {'range': {'gte': 1509674368.0, 'lte': 1514498432.0}}],
  'temporal_resolution': 'hour'}]

In [72]:
df = pd.read_csv('datasets/yellow.csv')

In [81]:
df = pd.read_csv("datasets/yellow.csv")
temp=pd.to_datetime(df['tpep_pickup_datetime'],format='%Y-%m-%d %H:%M:%S')

In [85]:
temp

0        2017-07-01 00:00:00
1        2017-07-01 00:00:00
2        2017-07-01 00:00:00
3        2017-07-01 00:00:00
4        2017-07-01 00:00:00
                 ...        
526610   2017-12-31 23:00:00
526611   2017-12-31 23:00:00
526612   2017-12-31 23:00:00
526613   2017-12-31 23:00:00
526614   2017-12-31 23:00:00
Name: tpep_pickup_datetime, Length: 526615, dtype: datetime64[ns]

In [86]:
metadata=datamart_profiler.process_dataset("datasets/LAPD_Calls_for_Service_2014.csv")
metadata.keys()

dict_keys(['size', 'nb_rows', 'average_row_size', 'nb_profiled_rows', 'nb_columns', 'columns', 'nb_temporal_columns', 'nb_categorical_columns', 'nb_numerical_columns', 'types', 'temporal_coverage', 'attribute_keywords'])

In [87]:
metadata['temporal_coverage']

[{'type': 'datetime',
  'column_names': ['Dispatch Date'],
  'column_indexes': [3],
  'column_types': ['http://schema.org/DateTime'],
  'ranges': [{'range': {'gte': 1389052800.0, 'lte': 1398643200.0}},
   {'range': {'gte': 1399680000.0, 'lte': 1409097600.0}},
   {'range': {'gte': 1410134400.0, 'lte': 1419465600.0}}],
  'temporal_resolution': 'day'}]

In [94]:
from datetime import datetime, timedelta

def getTimeRange(timeCoverage):
    epoch = datetime(1970, 1, 1)  # Unix epoch start date
    min_start_date = None
    max_start_date = None
    
    for range_item in timeCoverage:
        start_timestamp = range_item['range']['gte']
        end_timestamp = range_item['range']['lte']

        start_date = epoch + timedelta(seconds=start_timestamp)
        end_date = epoch + timedelta(seconds=end_timestamp)
        
        if min_start_date is None or start_date < min_start_date:
            min_start_date = start_date
        
        if max_start_date is None or end_date > max_start_date:
            max_start_date = end_date
    
    return str(min_start_date.time()), str(max_start_date.date())

getTimeRange(metadata['temporal_coverage'][0]['ranges'])

('00:00:00', '2014-12-25')

In [96]:
df = pd.read_csv(   "datasets/LAPD_Calls_for_Service_2014.csv")
df[]

,Incident Number,Reporting District,Area Occurred,Dispatch Date,Dispatch Time,Call Type Code,Call Type Description
0,140212004623,127,Central,02/12/2014,18:11:47,415W,WOMAN
1,140212004304,1385,Newton,02/12/2014,18:11:44,415G,GRP
2,140212004626,1307,Newton,02/12/2014,18:11:41,594O,OFCR HLDG
3,140212004685,2015,Olympic,02/12/2014,18:11:29,1101,NARCOTIC ACTIVITY
4,140212004668,1636,Foothill,02/12/2014,18:10:58,904A,AMB
...,...,...,...,...,...,...,...
932776,141231006434,396,Southwest,12/31/2014,23:58:36,507R,RADIO
932777,141231006436,679,Hollywood,12/31/2014,23:56:48,507P,PARTY
932778,141231006443,1974,Mission,12/31/2014,23:59:48,246H,HEARD ONLY
932779,141231006447,1235,77th Street,12/31/2014,23:58:02,245FX,SHOTS FIRED I/P


In [97]:
df['Area Occurred'].value_counts()

77th Street    61241
Southwest      55341
Mission        53418
N Hollywood    51156
Van Nuys       47466
Hollywood      45841
Pacific        45358
Northeast      44294
Southeast      43879
Rampart        43139
West Valley    42727
Newton         42116
Foothill       41688
Olympic        41101
Topanga        41029
Wilshire       40089
Devonshire     39895
Harbor         39503
West LA        38268
Central        38073
Hollenbeck     37159
Name: Area Occurred, dtype: int64

In [95]:
metadata['columns']

[{'name': 'Incident Number',
  'structural_type': 'http://schema.org/Integer',
  'semantic_types': [],
  'unclean_values_ratio': 0.0,
  'num_distinct_values': 81702,
  'mean': 140667694247.15616,
  'stddev': 339178521.0635877,
  'coverage': [{'range': {'gte': 140107000331.0, 'lte': 140425001653.0}},
   {'range': {'gte': 140507001862.0, 'lte': 140825004031.0}},
   {'range': {'gte': 140907000796.0, 'lte': 141225002548.0}}]},
 {'name': 'Reporting District',
  'structural_type': 'http://schema.org/Integer',
  'semantic_types': [],
  'unclean_values_ratio': 0.0,
  'num_distinct_values': 1131,
  'mean': 1157.1637291620768,
  'stddev': 598.5313976248929,
  'coverage': [{'range': {'gte': 152.0, 'lte': 762.0}},
   {'range': {'gte': 837.0, 'lte': 1452.0}},
   {'range': {'gte': 1522.0, 'lte': 2155.0}}]},
 {'name': 'Area Occurred',
  'structural_type': 'http://schema.org/Text',
  'semantic_types': ['http://schema.org/Enumeration'],
  'num_distinct_values': 21},
 {'name': 'Dispatch Date',
  'struct